In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
import unicodedata
import re
import numpy as np
import os
import io

In [2]:
path_to_zip = keras.utils.get_file('spa-eng.zip', origin='https://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip', extract=True)

In [3]:
path_to_zip

'C:\\Users\\salim\\.keras\\datasets\\spa-eng_extracted'

In [4]:
path_to_file = os.path.join(os.path.dirname(path_to_zip), 'spa-eng_extracted', 'spa-eng', 'spa.txt')
path_to_file

'C:\\Users\\salim\\.keras\\datasets\\spa-eng_extracted\\spa-eng\\spa.txt'

In [5]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

In [6]:
print(unicode_to_ascii("سلام"))     
print(unicode_to_ascii("بَلالِ"))   
print(unicode_to_ascii("résumé"))  

سلام
بلال
resume


In [7]:
def preprocess_senetence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.~,])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.~,]+", " ", w)
    w = w.rstrip().strip()
    w = '<strat> ' + w + ' <end>'
    return w

In [8]:
en_sentence = "I have some friends to help."
preprocess_senetence(en_sentence)

'<strat> i have some friends to help . <end>'

In [9]:
sp_sentence = "Aún no he dicho nada."
preprocess_senetence(sp_sentence)

'<strat> aun no he dicho nada . <end>'

In [10]:
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
    word_pairs = [[preprocess_senetence(w) for w in l.split('\t')] for l in lines[:num_examples]]
    # print(word_pairs)
    return zip(*word_pairs)

In [11]:
print(create_dataset(path_to_file, 5))

In [12]:
en, sp = create_dataset(path_to_file, None)

In [13]:
len(en)

118964

In [14]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [15]:
def tokenize(lang):
    lang_tokenizer = keras.preprocessing.text.Tokenizer(filters='')
    lang_tokenizer.fit_on_texts(lang)
    tensor = lang_tokenizer.texts_to_sequences(lang)
    tensor = keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    return tensor, lang_tokenizer

In [16]:
def load_dataset(path, num_examples=None):
    targ_lang, inp_lang = create_dataset(path, num_examples)
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, target_lang_tokenizer = tokenize(targ_lang)
    return input_tensor, target_tensor, inp_lang_tokenizer, target_lang_tokenizer

In [17]:
input_tensor, target_tensor, input_lang_tokenizer, target_lang_tokenizer = load_dataset(path_to_file, 20000)

In [18]:
input_tensor

array([[  1,  97,   3, ...,   0,   0,   0],
       [  1, 179,   3, ...,   0,   0,   0],
       [  1, 466,   3, ...,   0,   0,   0],
       ...,
       [  1,   7,  10, ...,   0,   0,   0],
       [  1,   7,  10, ...,   0,   0,   0],
       [  1,   7,  10, ...,   0,   0,   0]],
      shape=(20000, 16), dtype=int32)

In [19]:
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(input_tensor, target_tensor, test_size=0.2, random_state=400)

In [21]:
def convert(lang, tensor):
    for t in tensor:
        if t != 0:
            print(t, ' .... ', lang.index_word[t])

In [22]:
convert(input_lang_tokenizer, input_tensor[0])

1  ....  <strat>
97  ....  ve
3  ....  .
2  ....  <end>


In [23]:
input_tensor[0]

array([ 1, 97,  3,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
      dtype=int32)

In [24]:
target_lang_tokenizer.word_index

{'<strat>': 1,
 '<end>': 2,
 '.': 3,
 'i': 4,
 'you': 5,
 '?': 6,
 'tom': 7,
 'is': 8,
 'it': 9,
 's': 10,
 'a': 11,
 'he': 12,
 't': 13,
 'the': 14,
 'we': 15,
 'm': 16,
 'me': 17,
 're': 18,
 'that': 19,
 'this': 20,
 'to': 21,
 'do': 22,
 'are': 23,
 'can': 24,
 'my': 25,
 'they': 26,
 'was': 27,
 'she': 28,
 'don': 29,
 'have': 30,
 'your': 31,
 'go': 32,
 'what': 33,
 'in': 34,
 'not': 35,
 'll': 36,
 'like': 37,
 'here': 38,
 'on': 39,
 'him': 40,
 'let': 41,
 'be': 42,
 'did': 43,
 'know': 44,
 'come': 45,
 'up': 46,
 'am': 47,
 'want': 48,
 'how': 49,
 ',': 50,
 'mary': 51,
 'get': 52,
 'who': 53,
 'very': 54,
 'now': 55,
 'need': 56,
 'has': 57,
 'please': 58,
 'no': 59,
 'there': 60,
 'help': 61,
 'her': 62,
 'love': 63,
 'at': 64,
 'see': 65,
 'just': 66,
 'out': 67,
 'his': 68,
 've': 69,
 'got': 70,
 'for': 71,
 'where': 72,
 'look': 73,
 'stop': 74,
 'one': 75,
 'us': 76,
 'good': 77,
 'car': 78,
 'too': 79,
 'so': 80,
 'all': 81,
 'why': 82,
 'will': 83,
 'an': 84,
 'hom

In [25]:
BUFFER_SIZE = len(X_train)
BATCH_SIZE = 64
steps_per_epoch = len(X_train) // BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(input_lang_tokenizer.word_index) + 1
vocab_targ_size = len(target_lang_tokenizer.word_index) + 1

In [26]:
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [27]:
class Encoder(keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_size):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.enc_units = enc_units
        self.embedding = keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = keras.layers.GRU(self.enc_units, return_sequences=True, return_state=True)

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state=hidden)
        return output, state
    
    def initilize_hidden_state(self):
        return tf.zeros((self.batch_size, self.enc_units))

In [28]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE) 

In [29]:
encoder

<Encoder name=encoder, built=False>

In [30]:
simple_hidden = encoder.initilize_hidden_state()
simple_hidden

<tf.Tensor: shape=(64, 1024), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(64, 1024), dtype=float32)>

In [31]:
example_input_batch, example_target_batch = next(iter(dataset))

اینجا همون متد کال صدا زده میشه یعنی مدل فیت میشه

In [32]:
encoder(example_input_batch, simple_hidden)

(<tf.Tensor: shape=(64, 16, 1024), dtype=float32, numpy=
 array([[[ 1.84819044e-03, -2.73354119e-04,  4.81493212e-03, ...,
           4.74657770e-03, -3.15781846e-03,  1.09547726e-03],
         [ 1.51093351e-03,  7.82954041e-04,  1.05450815e-03, ...,
          -7.00181304e-03, -4.83156880e-04, -2.82977941e-03],
         [ 4.02191235e-03,  4.49710153e-03, -5.89088770e-03, ...,
          -5.31540718e-03,  7.27625506e-04,  1.80288672e-03],
         ...,
         [-2.90659582e-03,  3.13229440e-03, -8.13424028e-03, ...,
           1.08084446e-02, -9.11219232e-03,  1.85185391e-02],
         [-3.03838728e-03,  3.18629085e-03, -8.21223669e-03, ...,
           1.08615328e-02, -9.08510759e-03,  1.85532328e-02],
         [-3.11482744e-03,  3.22230067e-03, -8.25850386e-03, ...,
           1.08887050e-02, -9.07120109e-03,  1.85656715e-02]],
 
        [[ 1.84819044e-03, -2.73354119e-04,  4.81493212e-03, ...,
           4.74657770e-03, -3.15781846e-03,  1.09547726e-03],
         [-4.25209757e-04,  3.

In [33]:
simple_output, simple_states = encoder(example_input_batch, simple_hidden)

In [34]:
class Attention(keras.layers.Layer):
    def __init__ (self, units):
        super(Attention, self).__init__()
        self.W1 = keras.layers.Dense(units)
        self.W2 = keras.layers.Dense(units)
        self.V = keras.layers.Dense(1)
        
    def call (self, query, values):
        hidden_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))
        atteion_weights = tf.nn.softmax(score, axis=1)
        context_vector = atteion_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, atteion_weights

In [35]:
attention_layer = Attention(10)
attention_layer(simple_hidden, simple_output)

(<tf.Tensor: shape=(64, 1024), dtype=float32, numpy=
 array([[ 0.0015549 ,  0.00196846, -0.00438168, ...,  0.00536362,
         -0.00445573,  0.00952649],
        [-0.00014505,  0.00267691,  0.00034737, ...,  0.00969153,
         -0.00435913,  0.01024607],
        [ 0.00343958,  0.00546601, -0.00015206, ...,  0.0049861 ,
         -0.00804402,  0.00627997],
        ...,
        [-0.00201446,  0.00222688, -0.00405635, ...,  0.00520826,
         -0.00346711,  0.00872625],
        [ 0.00201516,  0.00036857, -0.00110448, ...,  0.00629374,
         -0.00398075,  0.00913824],
        [-0.00023378,  0.00472771, -0.0002429 , ...,  0.00480943,
         -0.00567034,  0.0075749 ]], shape=(64, 1024), dtype=float32)>,
 <tf.Tensor: shape=(64, 16, 1), dtype=float32, numpy=
 array([[[0.06144132],
         [0.06141929],
         [0.06212384],
         ...,
         [0.0628771 ],
         [0.0628839 ],
         [0.06288878]],
 
        [[0.06157065],
         [0.06078628],
         [0.06225225],
        

In [36]:
attention_result, attention_weights = attention_layer(simple_hidden, simple_output)

In [37]:
class Decoder(keras.Model):
    def __init__ (self, vocab_size, embedding_dim, dec_units, batch_size):
        super(Decoder, self).__init__()
        self.batch_size = batch_size
        self.dec_units = dec_units
        self.embedding = keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = keras.layers.GRU(self.dec_units, return_sequences=True, return_state=True)
        self.fc = keras.layers.Dense(vocab_size)
        self.attention = Attention(self.dec_units)
    def call(self, x, hidden, enc_output):
        context_vector, attention_weights = self.attention(hidden, enc_output)
        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        output, state = self.gru(x)
        output = tf.reshape(output, (-1, output.shape[2]))
        x = self.fc(output)
        return x, state, attention_weights

In [38]:
decoder = Decoder(vocab_targ_size, embedding_dim, units, BATCH_SIZE)

In [39]:
decoder(tf.random.uniform((BATCH_SIZE, 1)), simple_hidden, simple_output)

(<tf.Tensor: shape=(64, 3727), dtype=float32, numpy=
 array([[ 1.9888279e-03, -3.9440597e-04, -1.0000544e-03, ...,
         -6.5697427e-04, -2.9546330e-03,  2.9806781e-03],
        [ 1.8500288e-03,  2.1690939e-04, -4.9928506e-04, ...,
         -9.0362399e-04, -2.9271138e-03,  2.9839682e-03],
        [ 2.5576528e-03,  4.9895630e-04, -1.2741254e-03, ...,
         -3.1200133e-04, -2.0842957e-03,  2.6676375e-03],
        ...,
        [ 1.7599361e-03,  5.2244292e-04, -2.3244199e-04, ...,
         -1.4585352e-03, -3.1376907e-03,  2.9157675e-03],
        [ 2.3150202e-03,  5.6590070e-05, -8.0309011e-04, ...,
         -1.2324005e-03, -2.9551946e-03,  3.2195034e-03],
        [ 2.3689768e-03,  6.4942054e-04, -5.2455964e-04, ...,
         -1.0549023e-03, -3.2236073e-03,  2.6269245e-03]],
       shape=(64, 3727), dtype=float32)>,
 <tf.Tensor: shape=(64, 1024), dtype=float32, numpy=
 array([[ 0.00093429, -0.00210999,  0.00100665, ...,  0.00597961,
          0.00039489, -0.00394778],
        [ 0.0008

In [40]:
optimizer = keras.optimizers.Adam()
loss_object = keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

In [41]:
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

In [42]:
checkpoint_dir = 'chckpnts'
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder)

In [43]:
def train_step(inp, targ, enc_hidden):
    loss = 0
    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([target_lang_tokenizer.word_index['<strat>']] * BATCH_SIZE, 1)
        for t in range(1, targ.shape[1]):
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
            loss += loss_function(targ[:, t], predictions)
            dec_input = tf.expand_dims(targ[:, t], 1)
    batch_loss = (loss / int(targ.shape[1]))
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    return batch_loss

In [44]:
EPOCH = 7
for epoch in range(EPOCH):
    enc_hidden = encoder.initilize_hidden_state()
    total_loss = 0
    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss
        print('Epoch: ', epoch)
        print('Loss: ', batch_loss.numpy())
    checkpoint.save(file_prefix='chp/test1')

Epoch:  0
Loss:  4.7539096
Epoch:  0
Loss:  4.718105
Epoch:  0
Loss:  4.503395
Epoch:  0
Loss:  3.9899757
Epoch:  0
Loss:  3.371056
Epoch:  0
Loss:  3.1211824
Epoch:  0
Loss:  2.9368458
Epoch:  0
Loss:  2.8873913
Epoch:  0
Loss:  2.7312312
Epoch:  0
Loss:  2.7988856
Epoch:  0
Loss:  2.737929
Epoch:  0
Loss:  2.8631697
Epoch:  0
Loss:  2.7692966
Epoch:  0
Loss:  2.7470663
Epoch:  0
Loss:  2.7524889
Epoch:  0
Loss:  2.619088
Epoch:  0
Loss:  2.6877997
Epoch:  0
Loss:  2.7160268
Epoch:  0
Loss:  2.6671548
Epoch:  0
Loss:  2.6003487
Epoch:  0
Loss:  2.528337
Epoch:  0
Loss:  2.6207774
Epoch:  0
Loss:  2.7304873
Epoch:  0
Loss:  2.591435
Epoch:  0
Loss:  2.5190375
Epoch:  0
Loss:  2.3987846
Epoch:  0
Loss:  2.50766
Epoch:  0
Loss:  2.5926058
Epoch:  0
Loss:  2.4003062
Epoch:  0
Loss:  2.5822635
Epoch:  0
Loss:  2.5544038
Epoch:  0
Loss:  2.4741187
Epoch:  0
Loss:  2.400056
Epoch:  0
Loss:  2.4133217
Epoch:  0
Loss:  2.3439043
Epoch:  0
Loss:  2.3692436
Epoch:  0
Loss:  2.328614
Epoch:  0
Lo

In [45]:
def evaluate(sentence):
    sentence = preprocess_senetence(sentence)
    inputs = [input_lang_tokenizer.word_index[i] for i in sentence.split(' ')]
    inputs = keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    result = ''
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([target_lang_tokenizer.word_index['<strat>']], 0)
    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        attention_weights = tf.reshape(attention_weights, (-1, ))
        predicted_id = tf.argmax(predictions[0]).numpy()
        result += target_lang_tokenizer.index_word[predicted_id] + ' '
        if target_lang_tokenizer.index_word[predicted_id] == '<end>':
            return result, sentence
        dec_input = tf.expand_dims([predicted_id], 0)
    return result, sentence

In [46]:
checkpoint.restore(tf.train.latest_checkpoint('chp/'))

In [47]:
evaluate('hace mucho frio aqui.')

('it s very hot . <end> ', '<strat> hace mucho frio aqui . <end>')